In [1]:
import numpy as np
import pandas as pd

In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [3]:
COLUMN_DICT = {
    '日增长率': 'rzdf',
    '近一周': 'zzf',
    '近一月': '1yzf',
    '近3月': '3yzf',
    '近6月': '6yzf',
    '近1年': '1nzf',
    '近2年': '2nzf',
    '近3年': '3nzf',
    '今年以来': 'jnzf',
    '成立以来': 'lnzf'
}

COLS = [
    '代码', '名称', '简拼', '更新日期', '单位净值', '累积净值', '日增长率', '近一周', '近一月', '近3月',
    '近6月', '近1年', '近2年', '近3年', '今年以来', '成立以来', '成立日期'
]

In [4]:
def make_funds_rank_url(order_item):
    '''根据排名项生成获取基金排名数据的url'''
    return 'http://fund.eastmoney.com/data/rankhandler.aspx?op=ph&sc={}&st=desc'.format(
        COLUMN_DICT[order_item])


def get_funds_rank(url,n):
    '''获取基金排名数据'''
    r = requests.get(url)
    r.encoding = 'utf-8'
    datas = eval(re.findall('\[.*\]', r.text)[0])
    if n!=0:
        datas=datas[:n]
    return datas


def make_funds_df(datas):
    '''根据基金排名数据生成DataFrame'''
    funds = []
    for data in datas:
        funds.append(data.split(',')[:17])

    df = pd.DataFrame(funds, columns=COLS)
    return df


def get_funds_df(order_item,n):
    '''根据指定排名项获取基金排名数据的DataFrame'''
    url = make_funds_rank_url(order_item)
    datas = get_funds_rank(url,n)
    df = make_funds_df(datas)
    return df


def get_stocks_by_fund(fund):
    '''根据单个基金获取持仓股票信息'''
    url = 'http://fund.eastmoney.com/{}.html'.format(fund['代码'])
    r = requests.get(url)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text, 'lxml')

    # tag为每个股票的链接html
    i = 0
    stocks = []
    for tag in soup.select('#position_shares td.alignLeft a'):
        i += 1
        stocks.append([
            i, fund['代码'], fund['名称'],
            re.findall('\d+\d', tag['href'])[0], tag['title'], tag['href']
        ])
    return stocks


def get_stocks_df(order_item,n=0):
    '''获取所有基金的持仓股票信息'''
    funds_df = get_funds_df(order_item,n)
    
    stocks = []
    for index, fund in funds_df.iterrows():
        stocks_by_fund = get_stocks_by_fund(fund)
        stocks.extend(stocks_by_fund)
        
    stocks_df = pd.DataFrame(stocks,
                             columns=[
                                 'rank', 'fund_code', 'fund_name',
                                 'stock_code', 'stock_name', 'stock_url'
                             ])
    return stocks_df

In [5]:
order_item = '今年以来'

In [9]:
df=get_stocks_df(order_item,0)

In [75]:
df.drop_duplicates(['fund_code','fund_name'])

,rank,fund_code,fund_name,stock_code,stock_name,stock_url
0,1,519674,银河创新成长混合,002241,歌尔股份,http://quote.eastmoney.com/sz002241.html
10,1,007300,国联安中证半导体ETF联接A,603160,汇顶科技,http://quote.eastmoney.com/sh603160.html
20,1,007301,国联安中证半导体ETF联接C,603160,汇顶科技,http://quote.eastmoney.com/sh603160.html
30,1,000522,华润元大信息传媒科技混合,300308,中际旭创,http://quote.eastmoney.com/sz300308.html
40,1,004666,长城久嘉创新成长混合,603986,兆易创新,http://quote.eastmoney.com/sh603986.html
50,1,001956,国联安科技动力,300661,圣邦股份,http://quote.eastmoney.com/sz300661.html
60,1,005311,万家经济新动能混合A,300223,北京君正,http://quote.eastmoney.com/sz300223.html
70,1,005312,万家经济新动能混合C,300223,北京君正,http://quote.eastmoney.com/sz300223.html
80,1,257070,国联安优选行业混合,603986,兆易创新,http://quote.eastmoney.com/sh603986.html
90,1,001210,天弘互联网混合,002475,立讯精密,http://quote.eastmoney.com/sz002475.html


In [11]:
df.groupby(['stock_code','stock_name','stock_url']).count().sort_values('rank',ascending=False)

,,,rank,fund_code,fund_name
stock_code,stock_name,stock_url,,,
603986,兆易创新,http://quote.eastmoney.com/sh603986.html,35,35,35
002371,北方华创,http://quote.eastmoney.com/sz002371.html,18,18,18
603160,汇顶科技,http://quote.eastmoney.com/sh603160.html,15,15,15
600584,长电科技,http://quote.eastmoney.com/sh600584.html,15,15,15
000063,中兴通讯,http://quote.eastmoney.com/sz000063.html,15,15,15
002241,歌尔股份,http://quote.eastmoney.com/sz002241.html,14,14,14
002475,立讯精密,http://quote.eastmoney.com/sz002475.html,14,14,14
600703,三安光电,http://quote.eastmoney.com/sh600703.html,12,12,12
603501,韦尔股份,http://quote.eastmoney.com/sh603501.html,12,12,12


In [20]:
df_group=df['rank'].groupby([df['stock_code'],df['stock_name']])

In [91]:
x=df_group.size()

In [92]:
x[x.values>5].sort_values(ascending=False)

stock_code  stock_name
603986      兆易创新          35
002371      北方华创          18
000063      中兴通讯          15
603160      汇顶科技          15
600584      长电科技          15
002475      立讯精密          14
002241      歌尔股份          14
603501      韦尔股份          12
600703      三安光电          12
002156      通富微电          11
300661      圣邦股份          10
300433      蓝思科技           8
002273      水晶光电           8
002384      东山精密           7
300750      宁德时代           7
300782      卓胜微            7
300136      信维通信           6
688023      安恒信息           6
002185      华天科技           6
600845      宝信软件           6
688111      金山办公           6
Name: rank, dtype: int64

In [93]:
x=x[x.values>5].sort_values(ascending=False)

In [94]:
x

stock_code  stock_name
603986      兆易创新          35
002371      北方华创          18
000063      中兴通讯          15
603160      汇顶科技          15
600584      长电科技          15
002475      立讯精密          14
002241      歌尔股份          14
603501      韦尔股份          12
600703      三安光电          12
002156      通富微电          11
300661      圣邦股份          10
300433      蓝思科技           8
002273      水晶光电           8
002384      东山精密           7
300750      宁德时代           7
300782      卓胜微            7
300136      信维通信           6
688023      安恒信息           6
002185      华天科技           6
600845      宝信软件           6
688111      金山办公           6
Name: rank, dtype: int64

In [73]:
df[df['stock_name'].isin(x.index)].reset_index().groupby(['stock_name']).sum().sort_values('index')

,index,rank
stock_name,,
卓胜微,879,36
宝信软件,1025,41
安恒信息,1124,50
金山办公,1173,29
宁德时代,1264,61
信维通信,1749,45
水晶光电,1785,63
蓝思科技,1828,46
圣邦股份,2005,45


In [85]:
df[df['stock_name']=='通富微电']

,rank,fund_code,fund_name,stock_code,stock_name,stock_url
3,4,519674,银河创新成长混合,002156,通富微电,http://quote.eastmoney.com/sz002156.html
95,6,001210,天弘互联网混合,002156,通富微电,http://quote.eastmoney.com/sz002156.html
117,8,002844,金鹰多元策略混合,002156,通富微电,http://quote.eastmoney.com/sz002156.html
197,8,162703,广发小盘成长混合(LOF),002156,通富微电,http://quote.eastmoney.com/sz002156.html
264,5,000063,长盛电子信息主题灵活配置混合,002156,通富微电,http://quote.eastmoney.com/sz002156.html
272,3,001410,信达澳银新能源产业股票,002156,通富微电,http://quote.eastmoney.com/sz002156.html
376,7,001809,中信建投智信物联网A,002156,通富微电,http://quote.eastmoney.com/sz002156.html
386,7,004636,中信建投智信物联网C,002156,通富微电,http://quote.eastmoney.com/sz002156.html
395,6,006257,信达澳银先进智造股票型,002156,通富微电,http://quote.eastmoney.com/sz002156.html
453,4,519642,银河智造混合,002156,通富微电,http://quote.eastmoney.com/sz002156.html


In [87]:
df[df['stock_name']=='太极实业']

,rank,fund_code,fund_name,stock_code,stock_name,stock_url
16,7,007300,国联安中证半导体ETF联接A,600667,太极实业,http://quote.eastmoney.com/sh600667.html
26,7,007301,国联安中证半导体ETF联接C,600667,太极实业,http://quote.eastmoney.com/sh600667.html
